# Prepare environment

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# clone repo via ssh

!cp /content/drive/MyDrive/.ssh /root/  -r
!chmod 644 /root/.ssh/known_hosts
!chmod 600 /root/.ssh/id_rsa
!ssh -T git@github.com

!git clone -q --recurse-submodules git@github.com:annapanfil/punctuation_prediction.git

In [ ]:
%cd punctuation_prediction/
!pip install -q -r requirements.txt

In [ ]:
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = "annapanfil"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "f3ab7a17321dd66221aae96b7dbc9f43434a812d"
# os.environ['TOKENIZERS_PARALLELISM'] = "true"

# Conduct experiments

In [ ]:
!python3 src/dataset_creation.py --newlines=false

## Done

In [ ]:
# models
sh = """
#!/bin/bash

# Function to run training
run_training() {
    pretrained_model=$1
    language=$2
    epochs=$3
    command="python src/train.py --pretrained-model=${pretrained_model} --language=${language} --epoch=${epochs} --cuda=True"
    echo "Running command: $command"
    $command
}

# List of different models to try
models=("herbert-base" "bert-multiling-uncased" "roberta-multiling" "polish-roberta")
language="polish"
epochs=10

# Run training for each model
for model in "${models[@]}"
do
    run_training "$model" "$language" "$epochs"
done
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

In [ ]:
# !python src/train.py --cuda=True --pretrained-model=bert-base-uncased --freeze-bert=False --lstm-dim=-1 --language=english --seed=1 --lr=5e-6 --epoch=8 --use-crf=False --augment-type=all  --augment-rate=0.15 --alpha-sub=0.4 --alpha-del=0.4 --data-path=data --save-path=out
# !python src/train.py --pretrained-model=herbert-base --language=polish --epoch=10

In [ ]:
# !python src/inference.py --pretrained-model=bert-base-uncased --weight-path=out/weights.pt --language=en --in-file=data/test_en.txt --out-file=data/test_en_out.txt
# !python src/inference.py --pretrained-model=herbert-base --weight-path=out/weights.pt --language=pl --in-file=data/test_pl.txt --out-file=data/test_pl_out.txt

In [ ]:
# LSTM and freezing BERT

!python src/train.py --freeze-bert=False --lstm=bi --name="Bi, no freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"
!python src/train.py --freeze-bert=True --lstm=bi --name="Bi, freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=20 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"

!python src/train.py --freeze-bert=False --lstm=uni --name="Uni, no freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"
!python src/train.py --freeze-bert=True --lstm=uni --name="Uni, freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=20 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"

!python src/train.py --freeze-bert=False --lstm=none --name="No LSTM, no freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"
!python src/train.py --freeze-bert=True --lstm=none --name="No LSTM, freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=20 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"

## TODO

In [ ]:
# hyperparameters

sh = """
#!/bin/bash

# Function to run training
run_training() {
    lr=$1
    name="lr=$lr"
    command="python src/train.py --name=${name} --lr=${lr} --epoch=10 --freeze-bert=False --lstm=none --language=polish --cuda=True --pretrained-model=polish-roberta --log=true --data-variation=allpunct --experiment-name=hyperparameters"
    echo "Running command: $command"
    $command
}

# List of different models to try
lrs=(1e-1 1e-2 1e-3 1e-4 1e-5)

# Run training for each model
for lr in "${lrs[@]}"
do
    run_training "$lr"
done
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

In [ ]:
!python src/train.py --name="No LSTM, no freeze" --experiment-name="LSTM and freezing BERT" --use-lora=True --batch-size=16 --epoch=10 --sequence-length=128 --lr=1e-5 --log=true --freeze-bert=False --lstm=none  --data-variation=allpunct --language=polish --cuda=True --pretrained-model=polish-roberta

In [ ]:
!python src/train.py --name="LORA r=128 batch 32" --experiment-name="hyperparameters" --use-lora=True --batch-size=32 --epoch=10 --sequence-length=128 --lr=1e-5 --freeze-bert=False --lstm=none --log=true --data-variation=allpunct --pretrained-model=polish-roberta --language=polish --cuda=True